# Validation test FLEXI
 

In [80]:
###########################################################################################################
# OBJECTIVE: Use SCPI with Python to connect to several instruments to perform the validation tests on a
# 4-switch buck-boost for the FLEXI project. 
# Input voltage range: 80-150
# Output voltage: 100 V
# Output current: 5 A
# switching frequency: 100kHz
#  
# sensors: Voltage (In-Out), Current (In-Out), Inductor current
###########################################################################################################

#  address values

In [81]:
import pyvisa
import time
import numpy as np
import pandas as pd
import sys

ExitMode=['exit','Exit','EXIT']
rm = pyvisa.ResourceManager()
resources=rm.list_resources()
instruments = np.array(resources)
print(resources)
devices = []
for resource in resources:
    try:
        instrument = rm.open_resource(resource)
#        print(f'{resource} is available')
        address = instrument.resource_name
        vendor = instrument.query("*IDN?").strip()
        devices.append([address, vendor, 'Available'])
        identity = instrument.query('*IDN?')
#        print(identity)
        instrument.close()
    except:
#        print(f'{resource} is not available')
        devices.append([resource,"","Not available"])

def classify_vendor(vendor):
    if 'MSO7034B' in vendor:
        return 'Oscilloscope'
    elif '34461' in vendor:
        return 'DMM'
    elif 'N3300' in vendor:
        return 'Load'
    elif 'AC6803' in vendor:
        return 'Input'
    else:
        return 'Other'
devices_df = pd.DataFrame(devices, columns=['Address', 'Vendor','Available?'])
devices_df['Function'] = devices_df['Vendor'].apply(classify_vendor)
dmm_mask = devices_df['Function'] == 'DMM'
other_mask = devices_df['Function'] != 'DMM'
dmm_addresses = (devices_df['Function'] == 'DMM') & (~devices_df['Address'].str.contains('MY57224092|MY57224113|MY57224258|MY57224696'))
                                
devices_df['Measure'] = np.where(dmm_mask & devices_df['Address'].str.contains('MY57224092'), 'Vin', 
                                np.where(dmm_mask & devices_df['Address'].str.contains('MY57224113'), 'Iin', 
                                         np.where(dmm_mask & devices_df['Address'].str.contains('MY57224258'), 'Vout',
                                                  np.where(dmm_mask & devices_df['Address'].str.contains('MY57224696'), 'Iout',np.nan))))




devices_df


('GPIB0::5::INSTR', 'USB0::0x0957::0x173D::MY50340340::0::INSTR', 'USB0::0x0957::0xAC07::JPWC002359::0::INSTR', 'USB0::0x2A8D::0x1301::MY57224092::0::INSTR', 'USB0::0x2A8D::0x1301::MY57224113::0::INSTR', 'USB0::0x2A8D::0x1301::MY57224258::0::INSTR', 'USB0::0x2A8D::0x1301::MY57224696::0::INSTR', 'USB0::0x2A8D::0x1401::MY57224258::0::INSTR')


,Address,Vendor,Available?,Function,Measure
0,GPIB0::5::INSTR,"Agilent Technologies,N3300A,0,A.00.08",Available,Load,nan
1,USB0::0x0957::0x173D::MY50340340::0::INSTR,"AGILENT TECHNOLOGIES,MSO7034B,MY50340340,06.15...",Available,Oscilloscope,nan
2,USB0::0x0957::0xAC07::JPWC002359::0::INSTR,"Agilent,AC6803A,JPWC002359,A.01.02.0088",Available,Input,nan
3,USB0::0x2A8D::0x1301::MY57224092::0::INSTR,"Keysight Technologies,34461A,MY57224092,A.02.1...",Available,DMM,Vin
4,USB0::0x2A8D::0x1301::MY57224113::0::INSTR,"Keysight Technologies,34461A,MY57224113,A.02.1...",Available,DMM,Iin
5,USB0::0x2A8D::0x1301::MY57224258::0::INSTR,,Not available,Other,nan
6,USB0::0x2A8D::0x1301::MY57224696::0::INSTR,"Keysight Technologies,34461A,MY57224696,A.02.1...",Available,DMM,Iout
7,USB0::0x2A8D::0x1401::MY57224258::0::INSTR,"Keysight Technologies,34461A,MY57224258,A.02.1...",Available,DMM,Vout


In [82]:
Test_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
rows=['PNG name','CSV name']
# Create a DataFrame with the columns defined by Test_sequence
df = pd.DataFrame(columns=Test_sequence,index=rows)
ChanNum=4
Oscil_Address= devices_df.loc[devices_df['Function'] == 'Oscilloscope', 'Address'].values[0]
VinDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Vin'), 'Address'].values[0]
VoutDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Vout'), 'Address'].values[0]
Load_Address = devices_df.loc[devices_df['Function'] == 'Load', 'Address'].values[0]
IinDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Iin'), 'Address'].values[0]
IoutDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Iout'), 'Address'].values[0]
Vin_Address = devices_df.loc[devices_df['Function'] == 'Input', 'Address'].values[0]



# Table with results

In [83]:
DMM_sequence=['Vinmeas','Iinmeas','Voutmeas','Ioutmeas','Pin','Pout','Eff']
Oscil_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
# Test_sequence = DMM_sequence+Oscil_sequence
Test_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
LoadCurr=['1','5'] # A
ConvMode=['Buck','Boost']
ConvModeVoltageDict={'Buck':'150','Boost':'80'}
files=['CSV', 'PNG']
MeasOsc={'Frequency':'FREQuency','Average':'VAVerage','Pk to pk':'VPP'}

ChanConfLevel = {}
for item in Test_sequence:
    if item in ['Iload', 'Iinput', 'Ioutput','Vinput sens','Iinput sens','Voutput sens','Ioutput sens']:
        ChanConfLevel[item] = ['5', '500mV', '2', '2']
    elif item in ['Vinput', 'Voutput']:
        ChanConfLevel[item] = ['50', '500mV', '2', '2']
    elif item in ['Vinput AC', 'Voutput AC']:
        ChanConfLevel[item] = ['200mV', '500mV', '2', '2']
    elif item in ['Iinput AC', 'Ioutput AC']:
        ChanConfLevel[item] = ['5', '500mV', '2', '0.5']
    elif item == 'Iload AC':
        ChanConfLevel[item] = ['5', '500mV', '1', '2']    
    elif item == 'Iload sens':
        ChanConfLevel[item] = ['50mV', '500mV', '2', '2']
    else:
        ChanConfLevel[item] = ['5', '500mV', '2', '2']

OffsetConf={}
for item in Test_sequence:
    if item in ['Iload', 'Iinput', 'Ioutput','Vinput sens','Iinput sens','Voutput sens','Ioutput sens']:
        OffsetConf[item] = ['0', '1.5', '6', '2']
    elif item in ['Vinput', 'Voutput']:
        OffsetConf[item] = ['50', '1.5', '6', '2']
    elif item in ['Vinput AC', 'Voutput AC']:
        OffsetConf[item] = ['0', '-500mV', '6', '2']
    elif item in ['Iinput AC', 'Ioutput AC']:
        OffsetConf[item] = ['0', '-500mV', '6', '0.5']
    elif item == 'Iload sens':
        OffsetConf[item] = ['50mV', '1.5', '6', '2']
    elif item == 'Iload AC':
        OffsetConf[item] = ['50mV', '1.5', '2', '2']
    else:
        OffsetConf[item] = ['5', '-500mV', '6', '2']

OnOffConf={}
for item in Test_sequence:
    if item in ['Vinput', 'Voutput', 'Vinput AC', 'Voutput AC']:
        OnOffConf[item]=['ON','OFF','ON','OFF']
    elif item in ['Iinput AC','Iinput', 'Ioutput', 'Ioutput AC']:
        OnOffConf[item]=['OFF','OFF','ON','ON']
    elif item in ['Iload','Iload AC']:
        OnOffConf[item]=['OFF','OFF','ON','OFF']
    elif item in ['Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']:     
        OnOffConf[item]=['OFF','ON','ON','OFF'] 

ACDCConf= {}
for item in Test_sequence:
    if item in ['Vinput AC', 'Voutput AC']:
        ACDCConf[item]=['AC','DC','DC','DC']
    elif item in ['Iinput AC', 'Ioutput AC']:
        ACDCConf[item]=['DC','DC','DC','AC']
    elif item in ['Iload sens']:
        ACDCConf[item]=['AC','DC','DC','DC']
    elif item in ['Iload AC']:
        ACDCConf[item]=['AC','DC','AC','DC']
    else:
        ACDCConf[item]=['DC','DC','DC','DC']

TrigConf={}
for item in Test_sequence:
    for val in LoadCurr:
        if item in ['Iload AC']:
            TrigConf[val,item]='0'
        else:
            TrigConf[val,item]=val



multiindex = pd.MultiIndex.from_product([ConvMode, LoadCurr], names=["ConvMode", "LoadCurr"])
df = pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([Oscil_sequence, files], names=["Measurement", "Files"]))

DMM_df=pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([DMM_sequence], names=["Measurement"]))

Meas_df=pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([Oscil_sequence,MeasOsc], names=["Measurement","Inductor current meas"]))

### Open connections

In [84]:
OutLoad = rm.open_resource(Load_Address)
Vin = rm.open_resource(Vin_Address)
Oscil=rm.open_resource(Oscil_Address)



# Oscilloscope measurements

In [85]:
###PRECONFIGURE LOAD????

OutLoad.write("*RST")
OutLoad.write("INP ON")
OutLoad.write("CHANnel 2")
OutLoad.write("FUNC CURR")
OutLoad.write("CURR 1")


for converter in ConvMode: #choose either buck or boost
###############--CONFIGURE INPUT--############################
    print('Configuring Input')

    Vin.write("*RST") #Reset to default settings
    Vin.write("OUTP:COUPLing DC")
    Vin.write("SOUR:VOLT:OFFS {}".format(ConvModeVoltageDict[converter]))
    Vin.write("SOUR:VOLTage:RANGE AUTO")
    Vin.write("SOUR:CURRent:OFFSET 8")
    vinmeas = Vin.query_ascii_values("MEAS:VOLT?")
    iinmeas = Vin.query_ascii_values("MEAS:CURR?")
    print("Voltage set is {}. Current limit is set to {}.".format(vinmeas[0],iinmeas[0]))

    for load_current in LoadCurr: # choose current
        print('Configuring Load')
        if input('Testing in {} mode at {} A. Press ENTER to continue or EXIT to cancel'.format(converter,load_current)) in ExitMode :
            sys.exit()

###############--CONFIGURE LOAD--############################
        OutLoad.write("INP ON")
        OutLoad.write("CHANnel 2") # Load channel we are using
        OutLoad.write("FUNC CURR")
        OutLoad.write("CURR {}".format(str(load_current[0])))
        Vin.write("OUTP 1") #Enable Vin source output AFTER configuring load (safety purposes)
        


###############--VERY IMPORTANT: START FPGA--##############
        input('START TURN ON SEQUENCE ON FPGA (switches). Verify it works. Press ENTER TO CONTINUE if all OK')
        GoOn=input('All OK? Should we start measuring? Press "y" to start')
        while GoOn!='y':
            GoOn=input('All OK? Should we start measuring? Press "y" to start')
        Vin.write("OUTP 1") #Enable Vin source output AFTER configuring load (safety purposes)    
        


###############--CONFIGURE DMM--#############################                
    # Configure Vin DMM
        try:    
            VinDMM = rm.open_resource(VinDMM_Address)
            VinDMM.write('DISP:TEXT "Vin Meas"')
            VinDMM.write("*RST")
            VinDMM.write("*CLS")
            VinDMM.write("CONF:VOLT:DC AUTO")
            VinDMM.write("TRIG:DEL:AUTO ON")
            VinDMM.query("*OPC?")
            Vinrcv=VinDMM.query("READ?")
            DMM_df.loc[(converter,load_current),'Vinmeas']=float(Vinrcv.replace('\n',''))
            
        except:
            print('Vin DMM not available')

        # Configure Iin DMM
        try:
            IinDMM = rm.open_resource(IinDMM_Address)
            IinDMM.write('DISP:TEXT "Iin Meas"')
            IinDMM.write("*RST")
            IinDMM.write("*CLS")
            IinDMM.write("CONF:CURR:DC AUTO")
            IinDMM.write("CONF:CURR:DC:TERMinals 10")
            IinDMM.write("TRIG:DEL:AUTO ON")
            IinDMM.query("*OPC?")
            Iinrcv=IinDMM.query("READ?")
            DMM_df.loc[(converter,load_current),'Iinmeas']=float(Iinrcv.replace('\n',''))
            
        except:
            print('Iin DMM not available')

        # Configure Vout DMM
        try:
            VoutDMM = rm.open_resource(VoutDMM_Address)
            VoutDMM.write('DISP:TEXT "Vout Meas"')
            VoutDMM.write("*RST")
            VoutDMM.write("*CLS")
            VoutDMM.write("CONF:VOLT:DC AUTO")
            VoutDMM.write("TRIG:DEL:AUTO ON")
            VoutDMM.query("*OPC?")
            Voutrcv=VoutDMM.query("READ?")
            DMM_df.loc[(converter,load_current),'Voutmeas']=float(Voutrcv.replace('\n',''))
            
        except:
            print('Vout DMM not available')

        # Configure Iout DMM
        try:
            IoutDMM = rm.open_resource(IoutDMM_Address)
            IoutDMM.write('DISP:TEXT "Iout Meas"')
            IoutDMM.write("*RST")
            IoutDMM.write("*CLS")
            IoutDMM.write("CONF:CURR:DC AUTO")
            IoutDMM.write("CONF:CURR:DC:TERMinals 10") # To choose 10 A range
            IoutDMM.write("TRIG:DEL:AUTO ON")
            IoutDMM.query("*OPC?")
            Ioutrcv=IoutDMM.query("READ?")
            DMM_df.loc[(converter,load_current),'Ioutmeas']=float(Ioutrcv.replace('\n',''))
            
        except:
            print('Iout DMM not available')

        for element in Oscil_sequence: # choose what to measure Vin, Vout, Etc
            test=input('We will be testing {}. Prepare the probes wherever it needs to be measured. Once ready, press ENTER to continue.'.format(element))       
            if test in ExitMode:
                sys.exit()
            else:
###############--CONFIGURE OSCILLOSCOPE--####################
                Oscil.write('*RST')
                Oscil.write("*CLS")
                ChanNum=4
                Oscil.write("TIMebase:MODE MAIN")
                Oscil.write("TIMebase:POSition 0")
                Oscil.write("TIMebase:RANGe 20 us")
                Oscil.write("TIMebase:REFerence CENTer")
                Oscil.write("TRIGger:SWEep AUTO")
                Oscil.write("TRIGger:NREJect 1")
                Oscil.write("TRIGger:HFReject 0")
                Oscil.write("TRIGger:MODE Edge")#Selects Trigger mode
                Oscil.write("TRIGger:EDGE:SOURce CHAN3")#Trigger source is channel 3 
                Oscil.write(":TRIG:EDGE:LEVEl {}".format(TrigConf[load_current,element]))#config trigger
                

                

                
                for i in range(ChanNum):
                    chan=i+1
                    Oscil.write(":CHAN{}:DISP {}".format(chan,OnOffConf[element][i]))
                    Oscil.write(":CHAN{}:BWLimit ON".format(chan))
                    Oscil.write(":CHAN{}:SCALe {}".format(chan,ChanConfLevel[element][i])) #config level
                    Oscil.write(":CHAN{}:OFFSet {}".format(chan,OffsetConf[element][i]))
                    Oscil.write(":CHAN{}:COUPling {}".format(chan,ACDCConf[element][i]))#config coupling
                    

                for mosc in MeasOsc:
                    Oscil.write("MEASUre:{} CHANnel3".format(MeasOsc[mosc]))
                    m=Oscil.query("MEASUre:{}? CHANnel3".format(MeasOsc[mosc]))
                    Meas_df.loc[(converter,load_current),(element,mosc)]=float(m.replace('\n',''))
                time.sleep(1)
                Oscil.write(":SAVE:FILEname '{}_{}'".format(converter,element))
                print("Saving file {}".format(element))
                NameSave = Oscil.query(":SAVE:FILEname?")
                print('Saving file as {}'.format(NameSave))
                
                Oscil.write(":SAVE:PWD '/usb0/prueba/'")
                print(Oscil.query(":SAVE:PWD?"))
         
                Oscil.write(":SAVE:IMAGe:FORMat PNG")
                Oscil.write(":SAVE:IMAGe:INKSaver OFF")
                Oscil.write(":SAVE:IMAGe:PALEtte COLOr")
                while not bool(Oscil.query("*opc?")): #wait until saving finished
                    pass
                time.sleep(3)
                Oscil.write(":SAVE:IMAGe:STARt {}".format(NameSave))
                df.loc[(converter,load_current),(element,'PNG')]=NameSave.replace('"','').replace('\n','')+'.png'
                while not bool(Oscil.query("*opc?")): #wait until saving finished
                    pass
                Oscil.write(":SAVE:WAVeform:FORMat CSV")
                Oscil.write(":SAVE:WAVeform:LENGth 1000")
                time.sleep(3)
                Oscil.write(":SAVE:WAVeform:STARt {}".format(NameSave))

                df.loc[(converter,load_current),(element,'CSV')]=NameSave.replace('"','').replace('\n','')+'.csv'

                while not bool(Oscil.query("*opc?")): #wait until saving finished
                    pass
 

          

        

           



Configuring Input
Voltage set is -0.0460592. Current limit is set to -0.0179393.
Configuring Load
Saving file Vinput
Saving file as "Vinput"

"/usb0/"

Saving file Vinput AC
Saving file as "Vinput AC"

"/usb0/"

Saving file Voutput
Saving file as "Voutput"

"/usb0/"

Saving file Voutput AC
Saving file as "Voutput AC"

"/usb0/"

Saving file Iload
Saving file as "Iload"

"/usb0/"

Saving file Iload AC
Saving file as "Iload AC"

"/usb0/"

Saving file Iinput
Saving file as "Iinput"

"/usb0/"

Saving file Iinput AC
Saving file as "Iinput AC"

"/usb0/"

Saving file Ioutput
Saving file as "Ioutput"

"/usb0/"

Saving file Ioutput AC
Saving file as "Ioutput AC"

"/usb0/"

Saving file Vinput sens
Saving file as "Vinput sens"

"/usb0/"

Saving file Iinput sens
Saving file as "Iinput sens"

"/usb0/"

Saving file Voutput sens
Saving file as "Voutput sens"

"/usb0/"

Saving file Ioutput sens
Saving file as "Ioutput sens"

"/usb0/"

Saving file Iload sens
Saving file as "Iload sens"

"/usb0/"

Config

SystemExit: 

c:\Users\PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Efficiency measurements

In [ ]:
DMM_df


Measurement           Vinmeas Iinmeas    Voutmeas Ioutmeas  Pin Pout  Eff
ConvMode LoadCurr                                                        
Buck     1         150.054541    -0.0  102.197903     -0.0  NaN  NaN  NaN
         5         149.963079    -0.0   99.185863      0.0  NaN  NaN  NaN
Boost    1                NaN     NaN         NaN      NaN  NaN  NaN  NaN
         5                NaN     NaN         NaN      NaN  NaN  NaN  NaN

### Close all connections

In [ ]:
Oscil.close()
VoutDMM.close()
IoutDMM.close()
IinDMM.close()
VinDMM.close()
OutLoad.close()
Vin.close()

In [ ]:
Meas_df

Measurement              Vinput                  Vinput AC                   \
Inductor current meas Frequency Average Pk to pk Frequency Average Pk to pk   
ConvMode LoadCurr                                                             
Buck     1             177900.0   0.893      2.5  178600.0    0.89     2.56   
         5             181200.0    4.88     2.56  179900.0    4.87     2.56   
Boost    1                  NaN     NaN      NaN       NaN     NaN      NaN   
         5                  NaN     NaN      NaN       NaN     NaN      NaN   

Measurement             Voutput                  Voutput AC  ... Iinput sens  \
Inductor current meas Frequency Average Pk to pk  Frequency  ...    Pk to pk   
ConvMode LoadCurr                                            ...               
Buck     1             178600.0   0.891      2.5   181200.0  ...         2.5   
         5             179200.0   4.875     2.56   179200.0  ...         NaN   
Boost    1                  NaN     NaN      NaN        NaN  ...         NaN   
         5                  NaN     NaN      NaN        NaN  ...         NaN   

Measurement           Voutput sens                  Ioutput sens          \
Inductor current meas    Frequency Average Pk to pk    Frequency Average   
ConvMode LoadCurr                                                          
Buck     1                179900.0   0.895      2.5     179200.0   0.894   
         5                     NaN     NaN      NaN          NaN     NaN   
Boost    1                     NaN     NaN      NaN          NaN     NaN   
         5                     NaN     NaN      NaN          NaN     NaN   

Measurement                    Iload sens                   
Inductor current meas Pk to pk  Frequency Average Pk to pk  
ConvMode LoadCurr                                           
Buck     1                 2.5   179900.0  -0.024     2.44  
         5                 NaN        NaN     NaN      NaN  
Boost    1                 NaN        NaN     NaN      NaN  
         5                 NaN        NaN     NaN      NaN  

[4 rows x 45 columns]

# Save all data


In [ ]:
devices_df.to_csv('C:\Users\PC\OneDrive - Universidad de Oviedo\PhD\5-FLEXI v2 validation\Prototype 1\test',index=True)
df.to_csv('C:\Users\PC\OneDrive - Universidad de Oviedo\PhD\5-FLEXI v2 validation\Prototype 1\test',index=True)
DMM_df.to_csv('C:\Users\PC\OneDrive - Universidad de Oviedo\PhD\5-FLEXI v2 validation\Prototype 1\test',index=True)
Meas_df.to_csv('C:\Users\PC\OneDrive - Universidad de Oviedo\PhD\5-FLEXI v2 validation\Prototype 1\test',index=True)

In [ ]:
Oscil.write("TIMebase:RANGe 50 us")

22